In [33]:
#This program loads the update client message type into its related transaction detail table inside of the blockchain db
import pandas as pd
import json
import psycopg2
import logging
import os

DB_NAME = 'blockchain'
DB_USER = 'postgres'
DB_PASSWORD = 'auth2020'
DB_HOST = 'localhost'
DB_PORT = '5432'


try:
    # Create a PostgreSQL connection
    conn = psycopg2.connect(
        dbname=DB_NAME, user=DB_USER, password=DB_PASSWORD, host=DB_HOST, port=DB_PORT
    )
    cur = conn.cursor()
    print("Connection established.")  # Log successful connection (you can modify this)

    # Configure logging to write to a log file
    logging.basicConfig(filename='log.txt', level=logging.INFO)
    logging.info("Connection established with Database.")  # Log the same message to the file


    # Additional code for database operations goes here...

except Exception as err:
    # Handle exceptions
    print("Unable to connect to database.")  # Log error (you can modify this)
    print(err)  # Print the specific error message

    # Optionally, write the error message to a file (error.txt)
    with open('error.txt', 'a') as error_file:
        error_file.write(str(err) + '\n')
        

Connection established.


In [34]:

def get_most_recent_transaction_revoke_id(conn):
    # Replace with your actual query to get the most recent transaction_revoke_id
    # Example: SELECT MAX(transaction_revoke_id) FROM your_table_name;
    try:
        with conn.cursor() as cursor:
            cursor.execute("SELECT transaction_revoke_id FROM transaction_header_revoke ORDER BY transaction_revoke_id DESC LIMIT 1;")
            result = cursor.fetchone()
            if result:
                return result[0]
            else:
                return None
    except psycopg2.Error as e:
        print(f"Error fetching data: {e}")
        return None

In [35]:

def main():
    block = '5921581'
    file_path = f'C:/Users/lmhmo/Indexer_Project/indexer_project/Decrypted_files/{block}.json'

    try:
        if os.path.exists(file_path):
            print(f"Found JSON file: {file_path}")

            with open(file_path, 'r') as json_file:
                json_data = json.load(json_file)

                conn = connect_to_database()
                transaction_revoke_id = get_most_recent_transaction_revoke_id(conn)

                if transaction_revoke_id:
                    insert_queries = []

                    for message in json_data['tx']['body']['messages']:
                        granter = message["granter"]
                        grantee = message["grantee"]

                        msg_insert_query = f"""
                            INSERT INTO transaction_detail_msg_revoke (transaction_revoke_id, granter, grantee)
                            VALUES ('{transaction_revoke_id}', '{granter}', '{grantee}');
                        """
                        insert_queries.append(msg_insert_query)

                    with conn.cursor() as cursor:
                        for query in insert_queries:
                            cursor.execute(query)
                        conn.commit()

                    print("Data inserted successfully!")
                else:
                    print("No recent transaction_revoke_id found.")
        else:
            print(f"JSON file not found: {file_path}")
    except psycopg2.Error as e:
        print(f"Error connecting to the database: {e}")

if __name__ == "__main__":
    main()


Found JSON file: C:/Users/lmhmo/Indexer_Project/indexer_project/Decrypted_files/5921581.json
Data inserted successfully!
